## Tensorflow exercises

In [1]:
%matplotlib inline
# standard library
import itertools

# pandas
import pandas as pd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# tensorflow
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# sklearn
import sklearn.datasets
from sklearn.preprocessing import OneHotEncoder

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [2]:
# constants
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
sess = tf.Session()
sess.run([node1, node2])  # returns the values of the given placeholders

[3.0, 4.0]

In [5]:
# create a computation graph
add_node = tf.add(node1, node2)
sess.run([add_node])  # added

[7.0]

In [6]:
# placeholders to input external variables
plh_a = tf.placeholder(tf.float32)
plh_b = tf.placeholder(tf.float32)
adder_node = plh_a + plh_b  # shorthand for tf.add

In [7]:
# feedng values via the second argument
print(sess.run(adder_node, {plh_a: 3, plh_b: 4.5}))  # the keys must be the tensors

# for sequences, output will be np.array
print(sess.run(adder_node, {plh_a: np.array([1, 3, 4]), plh_b: np.array([4, 5, 1])}))

7.5
[ 5.  8.  5.]


In [8]:
# variables are internal values that cand change
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

# must be initialized
init = tf.global_variables_initializer()

# they will run sequentially
sess.run(init)
print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [9]:
# compute MSE
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_mean(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

5.915


In [10]:
# optimize the graph, the dependant variables are optimized
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)  # the gradient is calculated automatically

sess.run(init) # set the variables to the default
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.97351521], dtype=float32), array([ 0.92213166], dtype=float32)]


### MNIST example

In [11]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [12]:
y_expect = tf.placeholder(tf.int32, [None, 10])

# single layer softmax regressor
# intialize the trainable variables
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# prediction func, softmax
y_predict= tf.nn.softmax(tf.matmul(x, W) + b)  # softmax lin comb

# loss
# we use the second one ,the first one is numreically unstable
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_predict), reduction_indices=[1]))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_predict, labels=y_expect)

# gradient descent
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

In [15]:
# un the session
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
tf.global_variables_initializer().run()

for i in range(2000):
    batch = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch[0], y_expect: batch[1]})

In [16]:
# compute accuracy
correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y_expect, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_expect: mnist.test.labels}))

0.9214
